## **IF MODULES ARE NOT INSTALLED , RUN THIS-->**

##### **Packages required : tensorflow, tensorflow_hub, numpy, opencv-python, matplotlib**

 ## **IF ALL OF THESE ARE INSTALLED RUN ABOVE FILES**

In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub

hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

In [2]:
import os
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

#### **CONVERSION OF VIDEO IF REQUIRED**

In [ ]:
def convert_to_720p(input_file, output_file):
    cap = cv2.VideoCapture(input_file)

    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    target_width = 1280
    target_height = 720

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, 30, (target_width, target_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            resized_frame = cv2.resize(frame, (target_width, target_height))
            out.write(resized_frame)
            cv2.imshow('Frame', resized_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

input_file = 'testvideo.mp4'
output_file = 'output_video_720p.mp4'

convert_to_720p(input_file, output_file)

### **CONVERT VIDEO TO STYLIZED FRAMES**

In [23]:
# INPUT VIDEO AND STYLE HERE


video_path = "testvideo.mp4"
output_folder = "temporary_video_frames"
stylized_output_folder = "stylized_frames_op"
style_img_path = "Vincent Van Gogh - The Starry Night.jpg"


# max_dim = 256, 512, 1024, 2048 --- change this for quality


def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)
    plt.show()

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)
user_defined_fps = 22

os.makedirs(output_folder, exist_ok=True)
os.makedirs(stylized_output_folder, exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('output_video.mp4', fourcc, user_defined_fps, (int(cap.get(3)), int(cap.get(4))))

style_image = load_img(style_img_path)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    if frame_count % int(fps / user_defined_fps) == 0:

        cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count}.jpg"), frame)
        content_image = load_img(os.path.join(output_folder, f"frame_{frame_count}.jpg"))
        
        # Perform stylization
        stylized_images = hub_model(tf.constant(content_image), tf.constant(style_image))
        stylized_image = tf.squeeze(stylized_images[0], axis=0)  # Remove extra dimension

        # Display content and stylized images for debugging
        # imshow(content_image[0], title='Content Image')
        # imshow(stylized_image, title='Stylized Image')
        # Convert to numpy array and save
        stylized_image_np = tf.cast(stylized_image * 255, tf.uint8).numpy()
        encoded_image = cv2.cvtColor(stylized_image_np, cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
        cv2.imwrite(os.path.join(stylized_output_folder, f"stylized_frame_{frame_count}.jpg"), encoded_image)
        # temp_style_image_path = os.path.join(stylized_output_folder, f"stylized_frame_{frame_count}.jpg")

        output_video.write(encoded_image)

    # Print progress bar
    progress = frame_count / total_frames
    bar_length = 40
    progress_bar = int(bar_length * progress)
    percent = round(progress * 100, 2)
    bar = '=' * progress_bar + '.' * (bar_length - progress_bar)
    print(f'\rProgress: [{bar}] {percent}% Complete', end='', flush=True)

cap.release()
output_video.release()

# Print completion message
print("\nVideo processing completed.")

23.976023976023978
131
Progress: [========================================] 100.0% Complete
Video processing completed.


### **CONVERT FRAMES BACK TO VIDEO**

In [24]:
import cv2
import os


stylized_images_dir = "stylized_frames_op"
image_files = sorted(
    [os.path.join(stylized_images_dir, file) for file in os.listdir(stylized_images_dir)],
    key=lambda x: os.path.getmtime(x)
)

first_image = cv2.imread(image_files[0])
height, width, _ = first_image.shape

output_video_path = "output_stylized_video3.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, user_defined_fps, (width, height))

for image_file in image_files:
    image = cv2.imread(image_file)
    output_video.write(image)

output_video.release()

print(f"Video saved as: {output_video_path}")

Video saved as: output_stylized_video3.mp4
